this notebook is for extracting individual crystals from the large h5 files, and converting to their own individual h5 file
* one file at a time -> potentially loop in the future
* assume Jonny format, all images stacked together

In [19]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from glob import glob
import seaborn as sns
import xesmf as xe
import pandas as pd
#import def_homebrew as hb ## homemade functions xox
from scipy.special import gamma
import netCDF4 as nc
from datetime import datetime, timedelta

import h5py ####
from PIL import Image
#from IPython.display import display #
#import cv2 # not working
import os

In [20]:
## files location
ds_loc = '/home/users/esree/data/2ds/'
hvps_loc = '/home/users/esree/data/hvps/'

#file of interest
file_name = 'Export_base220723153010.h5' # example file
f2ds = h5py.File(ds_loc+ file_name,'r') # open file

# break file into two - data + time
ds_image = f2ds['ImageData'] 
ds_time = f2ds['ImageTimes']
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
##### make xarray of useful time data #####
sec_since = ds_time[:,0]
pixel_slice = ds_time[:,1]
pix_sum = pixel_slice.cumsum(dtype = 'int')

## make useful datetime format (not seconds since midnight)
# using the file name for reference
date_str = file_name[11:17]
starting_date = datetime.strptime(date_str, '%y%m%d')
time_deltas = [timedelta(seconds=float(sec)) for sec in sec_since]
utc_time = [starting_date + delta for delta in time_deltas]

time_xr =xr.Dataset({
    'utc_time':utc_time,
    'pixel_slice': pixel_slice,
    'pix_sum': pix_sum})
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [39]:
## this is the code for splitting up the big h5 file -> individual images
save_path = '/gws/nopw/j04/dcmex/users/ezriab/2dprocessed/'
folder_name = f'flight_{file_name[11:23]}' # each flightset -> own folder

if not os.path.exists(save_loc+folder_name):
    os.makedirs(save_loc+folder_name)
    print("Folder created successfully!")
else:
    print("Folder already exists.")
save_loc = save_path+folder_name+'/'

Folder created successfully!


In [45]:
# possibly 2 images are captured simultaneously, so will have to capture this in the name when extracting i.e. _0 or _1.h5
# this is a function to extract the image data + save appropriately

def extract_save_image(x, save_location, int='0'):
    im_s_idx = time_xr['pix_sum'][x].values
    im_e_idx = time_xr['pix_sum'][x+1].values

    one_crystal = ds_image[:,im_s_idx:im_e_idx]
    
    crystal_str = str(time_xr['utc_time'][im_s_idx].values)
    individual_file = f'{crystal_str[5:7]}{crystal_str[8:10]}-{crystal_str[11:13]}{crystal_str[14:16]}{crystal_str[17:19]}{crystal_str[20:22]}_{int}.h5'

    save_name = save_location+individual_file

    print(save_name)
    
    with h5py.File(save_name, 'w') as f:
    # Create a dataset in the HDF5 file
        f.create_dataset('data', data=one_crystal)


for i in range(100):
    if i = 0:
        extract_save_image
    elif time_xr['utc_time'][i] == time_xr['utc_time'][i-1]:
        # this account for repeat in time for two seperate images
        extract_save_image(i,save_loc,int='1')

    else:
        extract_save_image(i,save_loc)


18
/gws/nopw/j04/dcmex/users/ezriab/2dprocessed/flight_220723153010/0723-17511102_0.h5


"\n#print(one_crystal)\nfor i in range(50):\n    print(int(time_xr['pix_sum'][i]))\n\nfor i in range(20):\n    print(sec_since[i])\n\n"